In [ ]:
import os
import librosa
import math
import json

DATASET_PATH = "dataset_birdyy"
JSON_PATH = "birdyy.json"
SAMPLE_RATE = 44100
DURATION = 5
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

def mfcc_transformation(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=1024, num_segments=10):
    
    # slovník na ulozenie dat 
    data = {
        "mapping" : [],
        "mfcc" : [],
        "labels" : []
    }
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)
    
    # prechod cez vsetky priecinky v trenovacom datasete
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        
        if dirpath is not dataset_path:
            
            # extrakcia kategorii a ich ulozenie do slovnika
            dirpath_components = dirpath.split("/")
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing {}".format(semantic_label))
            
            # spracovanie suborov jednotlivych kategorii
            for f in filenames:
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
                
                # mfcc transformacia segmentov a ulozenie hodnot do slovnika
                for s in range(num_segments):
                    start_sample = num_samples_per_segment * s
                    finish_sample = start_sample + num_samples_per_segment
                    
                    mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample], 
                                                sr=SAMPLE_RATE, 
                                                n_fft=n_fft, 
                                                n_mfcc=n_mfcc, 
                                                hop_length=hop_length)
                    mfcc = mfcc.T
                    
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path,s))
    
    # ulozenie dat vo formate json
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

In [ ]:
mfcc_transformation(DATASET_PATH, JSON_PATH)